In [ ]:
import numpy as np

In [ ]:
def normal(x,theta):
    mu=theta[0]
    s=theta[1]#标准差
    result=(1/(np.sqrt(2*np.pi)*s))*np.exp(-1*(x-mu)**2/(2*s**2))
    return result

def mixnormal(x,theta):
    m=theta.shape[0]
    result=0
    for i in range(m):
        result+=theta[0,1]*normal(x,theta[1:,i])
    return result

In [ ]:
def Lih(X,theta):
    num=X.shape[0]
    C=theta.shape[1]
    Ga=np.zeros((num,C))
    c=np.zeros(C)

    for j in range(num):#响应度
        for k in range(C):
            c[k]=theta[0,k]*normal(X[j],theta[1:,k])
        Ga[j]=c/np.sum(c)

    return Ga

In [ ]:
def ECM(Xs,Y,betaold,thetaold,theta0,tp=0.25):
    P=Xs.shape[1]
    num=Xs.shape[0]
    data=Y-np.dot(Xs,betaold.T)
    L0=Lih(data,thetaold)
    C=thetaold.shape[1]
    d=np.zeros(C)
    thetanext=np.zeros((3,C))

    ai=np.zeros(num)
    bi=np.zeros(num)
    for i in range (num):
        for h in range(C):
            bi[i]+=(L0[i,h])/((thetaold[2,h])**2)
            ai[i]+=((L0[i,h])*((Y[i]-thetaold[1,h])))/((thetaold[2,h])**2)

    xnew=np.zeros((num,P))
    for i in range(num):
        xnew[i]=Xs[i]*bi[i]

    betanew=np.dot(np.dot(np.linalg.inv(np.dot(xnew.T,Xs)),Xs.T),ai)

    datanew=Y-np.dot(Xs,betanew.T)
    for k in range(C):#更新参数
        d[k]=np.sum(L0[:,k])
        thetanext[0,k]=d[k]/num
        thetanext[1,k]=np.dot(L0[:,k],datanew[:])/d[k]
        thetanext[2,k]=np.sqrt((np.dot(L0[:,k],(datanew[:]-thetanext[1,k])**2)+2*tp*theta0[2,k])/(d[k]+2*tp))#标准差

    return betanew,thetanext


def iterECM(Xs,Y,beta0,theta0,tp=0.25,N=1000,ε=0.005):
    T0=[beta0,theta0]
    for i in range(N):
        a=T0[0]
        T0=ECM(Xs,Y,T0[0],T0[1],theta0,tp)
        b=np.linalg.norm(T0[0]-a)
        '''
        if b<=ε:
            break
        '''
    return T0,b,i

In [ ]:
def ps(x,y,an=0.25):#对方差的惩罚函数
    z=-an*(np.log(x**2/y**2)+(y**2/x**2)-1)
    return z

def pa(α,C=1):#对系数的惩罚函数
    z=C*np.log(1-abs(1-2*α))
    return z

def ln(x,t1,t2,α):#单个样本的混合正态的对数似然
    n=x.shape[0]
    y=np.zeros(n)
    for i in range(n):
        y[i]=np.log(mixnormal(x[i],t1,t2,α))
    return y

def pl(x,t1,t2,α):#混合正态的惩罚联合对数似然
    y=np.sum(ln(x,t1,t2,α))+ps(t1[1],svar)+ps(t2[1],svar)+pa(α)
    return y

def ln0(x,th):#原假设下的惩罚对数似然
    n=x.shape[0]
    y=np.zeros(n)
    for i in range(n):
        y[i]=np.log(normal(x[i],th))
    z=np.sum(y)
    return z